In [40]:
import logging
import os
import sys

import numpy as np
import pandas as pd
import torch
from ignite.engine import Events, create_supervised_evaluator, create_supervised_trainer
from ignite.handlers import EarlyStopping, ModelCheckpoint
from ignite.metrics import Accuracy

import monai
import tensorboard
from monai.data import ImageDataset, decollate_batch, DataLoader
from monai.handlers import StatsHandler, TensorBoardStatsHandler, stopping_fn_from_metric
from monai.transforms import EnsureChannelFirst, Compose, RandRotate90, Resize, ScaleIntensity

In [41]:
monai.config.print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

MONAI version: 1.2.0
Numpy version: 1.24.3
Pytorch version: 2.0.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: c33f1ba588ee00229a309000e888f9817b4f1934
MONAI __file__: /home/hpatel33/anaconda3/lib/python3.11/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.1.0
scikit-image version: 0.20.0
Pillow version: 9.4.0
Tensorboard version: 2.14.1
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.15.2+cu117
tqdm version: 4.65.0
lmdb version: 1.4.1
psutil version: 5.9.0
pandas version: 1.5.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: 4.29.2
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: 1.0.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies


In [42]:
#list of image paths
imgFolder = './LiverSegmentations'
images = []
for folders in os.listdir(imgFolder):
    for files in os.listdir(os.path.join(imgFolder, folders)):
        imgPath = os.path.join(imgFolder, folders, files)
        images.append(imgPath)

In [43]:
#extract labels
labelPath = "./Colorectal Liver Metastases Clinical data April 2023.xlsx"
data = pd.read_excel(labelPath, sheet_name="CRLM (n=168)")
rec_5yr = data['Recurrence (5y)']
rec_2yr = data['Recurrence (2y)']
data_id = data['Patient-ID']
data_label = pd.concat([data_id, rec_2yr, rec_5yr], axis=1)

In [44]:
print(data_label.head())

     Patient-ID  Recurrence (2y)  Recurrence (5y)
0  CRLM-CT-1001                0                0
1  CRLM-CT-1002                1                1
2  CRLM-CT-1003                0                0
3  CRLM-CT-1004                0                0
4  CRLM-CT-1005                0                1


In [45]:
images = sorted(images)

In [46]:
paths = []
for patients in data_label['Patient-ID']:
    for image in images:
        if image.find(patients) != -1:
            paths.append(image)

path_toadd = {'Path':paths}
paths_df = pd.DataFrame(path_toadd)
data_label = pd.concat([data_label, paths_df], axis=1)

In [47]:
labels_5yr = np.array(data_label['Recurrence (5y)'], dtype=np.int64)
labels_2yr = np.array(data_label['Recurrence (2y)'], dtype=np.int64)

In [128]:
from sklearn.model_selection import train_test_split

x_train2, x_val2, y_train2, y_val2 = train_test_split(data_label, labels_2yr, test_size=0.3)

In [129]:
print(len(x_train2))
print(len(x_val2))
print(len(x_test2))

117
51
34


In [151]:
import nibabel as nib
import matplotlib.pyplot as plt
min_z = 10000
for paths in data_label['Path']:
    test_load = nib.load(paths).get_fdata()
    if test_load.shape[2] < min_z:
        min_z = test_load.shape[2]

print(min_z)

15


In [177]:
# define transforms
train_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize((512, 512, 96)), RandRotate90()])
val_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize((512, 512, 96))])

In [178]:
# define image dataset, data loader
check_ds = ImageDataset(image_files=np.array(x_train2['Path']), labels=np.array(y_train2), transform=train_transforms)
check_loader = DataLoader(check_ds, batch_size=2, num_workers=2, pin_memory=torch.cuda.is_available())
im, label = monai.utils.misc.first(check_loader)
print(type(im), im.shape, label)

<class 'monai.data.meta_tensor.MetaTensor'> torch.Size([2, 1, 512, 512, 96]) tensor([1, 1])


In [179]:
# create DenseNet121, CrossEntropyLoss and Adam optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = monai.networks.nets.DenseNet264(spatial_dims=3, in_channels=1, out_channels=2).to(device)
loss = torch.nn.CrossEntropyLoss()
lr = 1e-5
opt = torch.optim.Adam(net.parameters(), lr)

In [180]:
# Ignite trainer expects batch=(img, label) and returns output=loss at every iteration,
# user can add output_transform to return other values, like: y_pred, y, etc.
trainer = create_supervised_trainer(net, opt, loss, device, False)

In [181]:

# adding checkpoint handler to save models (network params and optimizer stats) during training
checkpoint_handler = ModelCheckpoint("./runs_array/", "net", n_saved=10, require_empty=False)
trainer.add_event_handler(
    event_name=Events.EPOCH_COMPLETED, handler=checkpoint_handler, to_save={"net": net, "opt": opt}
)

In [182]:
# StatsHandler prints loss at every iteration and print metrics at every epoch,
# we don't set metrics for trainer here, so just print loss, user can also customize print functions
# and can use output_transform to convert engine.state.output if it's not loss value
train_stats_handler = StatsHandler(name="trainer", output_transform=lambda x: x)
train_stats_handler.attach(trainer)

In [183]:
# TensorBoardStatsHandler plots loss at every iteration and plots metrics at every epoch, same as StatsHandler
train_tensorboard_stats_handler = TensorBoardStatsHandler(output_transform=lambda x: x)
train_tensorboard_stats_handler.attach(trainer)

In [184]:
# set parameters for validation
validation_every_n_epochs = 1

In [185]:
metric_name = "Accuracy"
# add evaluation metric to the evaluator engine
val_metrics = {metric_name: Accuracy()}
# Ignite evaluator expects batch=(img, label) and returns output=(y_pred, y) at every iteration,
# user can add output_transform to return other values
evaluator = create_supervised_evaluator(net, val_metrics, device, True)


In [186]:
# add stats event handler to print validation stats via evaluator
val_stats_handler = StatsHandler(
    name="evaluator",
    output_transform=lambda x: None,  # no need to print loss value, so disable per iteration output
    global_epoch_transform=lambda x: trainer.state.epoch,
)  # fetch global epoch number from trainer
val_stats_handler.attach(evaluator)

In [187]:
# add early stopping handler to evaluator
early_stopper = EarlyStopping(patience=4, score_function=stopping_fn_from_metric(metric_name), trainer=trainer)
evaluator.add_event_handler(event_name=Events.EPOCH_COMPLETED, handler=early_stopper)

In [188]:
# create a validation data loader
val_ds = ImageDataset(image_files=np.array(x_val2['Path']), labels=np.array(y_val2), transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=2, num_workers=2, pin_memory=torch.cuda.is_available())

In [189]:
@trainer.on(Events.EPOCH_COMPLETED(every=validation_every_n_epochs))
def run_validation(engine):
    evaluator.run(val_loader)

In [190]:
# create a training data loader
train_ds = ImageDataset(image_files=np.array(x_train2['Path']), labels=np.array(y_train2), transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=2, pin_memory=torch.cuda.is_available())

In [ ]:
train_epochs = 30
state = trainer.run(train_loader, train_epochs)
print(state)

INFO:ignite.engine.engine.Engine:Engine run starting with max_epochs=30.
2023-10-11 17:11:23,776 - INFO - Epoch: 1/30, Iter: 1/59 -- Loss: 0.5995 
2023-10-11 17:11:58,249 - INFO - Epoch: 1/30, Iter: 2/59 -- Loss: 0.8138 
2023-10-11 17:12:32,216 - INFO - Epoch: 1/30, Iter: 3/59 -- Loss: 0.8121 
2023-10-11 17:13:06,317 - INFO - Epoch: 1/30, Iter: 4/59 -- Loss: 0.6760 
2023-10-11 17:13:40,320 - INFO - Epoch: 1/30, Iter: 5/59 -- Loss: 0.6942 
2023-10-11 17:14:14,087 - INFO - Epoch: 1/30, Iter: 6/59 -- Loss: 0.7948 
2023-10-11 17:14:48,311 - INFO - Epoch: 1/30, Iter: 7/59 -- Loss: 0.7869 
2023-10-11 17:15:22,449 - INFO - Epoch: 1/30, Iter: 8/59 -- Loss: 0.6264 
2023-10-11 17:15:56,502 - INFO - Epoch: 1/30, Iter: 9/59 -- Loss: 0.5705 
2023-10-11 17:16:30,366 - INFO - Epoch: 1/30, Iter: 10/59 -- Loss: 0.7101 
2023-10-11 17:17:04,008 - INFO - Epoch: 1/30, Iter: 11/59 -- Loss: 0.6541 
2023-10-11 17:17:37,729 - INFO - Epoch: 1/30, Iter: 12/59 -- Loss: 0.6240 
2023-10-11 17:18:12,022 - INFO - Epo

{}
